In [8]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [ ]:
#          API VK  

In [1]:
token = 'dsfsdfhsd'

In [2]:
vk_session = vk_api.VkApi(token=token)
vk = vk_session.get_api()

NameError: name 'vk_api' is not defined

In [ ]:
vk.messages.send(
    chat_id=1,
    random_id=2,
    message='new message ...')

In [40]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-gazizova/shared/ads_data.csv.zip', compression='zip')
df.head()

,ad_id,time,event,date,ad_cost_type,has_video,client_union_id,campaign_union_id,platform,ad_cost,target_audience_count,user_id
0,23456,1554076848,view,2019-04-01,CPM,0,5681,23456,android,190.0,125560,793
1,111941,1554135866,view,2019-04-01,CPM,0,111793,111941,ios,215.2,32277,1041
2,111941,1554135866,view,2019-04-01,CPM,0,111793,111941,ios,215.2,32277,41
3,42987,1554135866,view,2019-04-01,CPM,0,28541,42985,ios,199.1,80160,898
4,23456,1554135866,view,2019-04-01,CPM,0,5681,23456,web,190.0,125560,29


In [39]:
ad_data = df.groupby(['ad_id', 'event'], as_index=False) \
    .agg({'user_id': 'count'})
ad_data

,ad_id,event,user_id
0,3,click,9
1,3,view,490
2,2132,click,1
3,2132,view,95
4,2276,click,2
...,...,...,...
656,122042,view,155
657,122063,click,1
658,122063,view,260
659,122086,click,50


In [41]:
ad_data = ad_data.pivot(index='ad_id', columns='event', values='user_id').reset_index()

In [42]:
ad_data = ad_data.fillna(0).assign(ctr=ad_data.click / ad_data.view)
ad_data

event,ad_id,click,view,ctr
0,3,9.0,490.0,0.018367
1,2132,1.0,95.0,0.010526
2,2276,2.0,1454.0,0.001376
3,2475,0.0,132.0,NaN
4,2643,3.0,286.0,0.010490
...,...,...,...,...
350,121941,1.0,640.0,0.001563
351,121943,15.0,1722.0,0.008711
352,122042,1.0,155.0,0.006452
353,122063,1.0,260.0,0.003846


In [43]:
top_ctr = ad_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)
top_ctr

event,ad_id,click,view,ctr
15,12904,32.0,399.0,0.080201
328,120347,236.0,2168.0,0.108856
5,4585,53.0,476.0,0.111345
96,35034,112.0,997.0,0.112337
194,45043,28.0,245.0,0.114286
207,45642,42.0,344.0,0.122093
144,40968,29.0,217.0,0.133641
125,38575,43.0,257.0,0.167315
324,119450,258.0,1254.0,0.205742
289,112583,580436.0,1934788.0,0.300000


In [44]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [ ]:
# отправка файла в чат в вк

path = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-gazizova/python_lessons/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)
json_data

In [ ]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)
saved_file

In [ ]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])
attachment

In [ ]:
vk.messages.send(
    chat_id=1,
    random_id=3,
    message='Топ объявлений по CTR', 
    attachment = attachment
)

In [ ]:
#           GOOGLE SPREADSHEETS API 

In [31]:
import pandas as pd
import requests
import json

import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [32]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

my_mail = 'darina.geo88@gmail.com'

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name('/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-gazizova/python_lessons/my-project-darina-5fc288213a02.json', scope)
gs = gspread.authorize(credentials)

In [33]:
# Name of the table in google sheets, 
# can be url for open_by_url
# or id (key) part for open_by_key
table_name = 'file_to_sequence'  # Your table
# Get this table
work_sheet = gs.open(table_name)

In [34]:
# Select 1st sheet
sheet1 = work_sheet.sheet1

# Get data in python lists format
data = sheet1.get_all_values()
data

[['price', 'group'],
 ['120', 'group 1'],
 ['20', 'group 2'],
 ['50', 'group 2'],
 ['30', 'group 2'],
 ['90', 'group 1'],
 ['200', 'group 3'],
 ['200', 'group 3'],
 ['300', 'group 4'],
 ['123', 'group 2']]

In [35]:
headers = data.pop(0)
headers

['price', 'group']

In [36]:
df = pd.DataFrame(data, columns=headers)
df

,price,group
0,120,group 1
1,20,group 2
2,50,group 2
3,30,group 2
4,90,group 1
5,200,group 3
6,200,group 3
7,300,group 4
8,123,group 2


In [26]:
# добавление данных в гугл-таблицу

sheet1.append_row([123, 'group 2'])

{'spreadsheetId': '127L2S-SRW3KKuMQ8imbfsMNzW-uYnvtKxUwhS7PEjKU',
 'tableRange': "'Лист1'!A1:B9",
 'updates': {'spreadsheetId': '127L2S-SRW3KKuMQ8imbfsMNzW-uYnvtKxUwhS7PEjKU',
  'updatedRange': "'Лист1'!A10:B10",
  'updatedRows': 1,
  'updatedColumns': 2,
  'updatedCells': 2}}

In [39]:
# Looks like spreadsheet should be already present at the dist (so, run code in create table section)

spreadsheet_name = 'A new spreadsheet'
sheet = '222'
d2g.upload(df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet '222' id:910747875>

In [45]:
# тест авторизации Google API и получения данных из Google Sheets после создания ключа Credentials


import gspread
from oauth2client.service_account import ServiceAccountCredentials


# Specify path to your file with credentials
path_to_credential = '/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-gazizova/python_lessons/my-project-darina-5fc288213a02.json' 

# Specify name of table in google sheets
table_name = 'file_to_sequence'

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_sheet = gs.open(table_name)   # открываем таблицу

# Select 1st sheet        -   выбирае нужный лист
sheet1 = work_sheet.sheet1

# Get data in python lists format  -   получаем данные
data = sheet1.get_all_values()

# Get header from data   
headers = data.pop(0)     # список заголовков

# Create df
df = pd.DataFrame(data, columns=headers)
df.head()

,price,group
0,120,group 1
1,20,group 2
2,50,group 2
3,30,group 2
4,90,group 1


In [ ]:
#           API Яндекс.Метрики

In [9]:
url = 'https://api-metrika.yandex.net/stat/v1/data?'
visits = 'metrics=ym:s:visits&dimensions=ym:s:date&id=44147844'

In [10]:
vistis_url = url + visits

In [12]:
vistis_request = requests.get(vistis_url)
vistis_request                          #  выгружаем

<Response [200]>

In [13]:
#  переводим в удобный формат

json_data = json.loads(vistis_request.text)
json_data

{'query': {'ids': [44147844],
  'dimensions': ['ym:s:date'],
  'metrics': ['ym:s:visits'],
  'sort': ['-ym:s:visits'],
  'date1': '2024-01-03',
  'date2': '2024-01-09',
  'limit': 100,
  'offset': 1,
  'group': 'Week',
  'auto_group_size': '1',
  'attr_name': '',
  'quantile': '50',
  'offline_window': '21',
  'attribution': 'LastSign',
  'currency': 'RUB',
  'adfox_event_id': '0'},
 'data': [{'dimensions': [{'name': '2024-01-08'}], 'metrics': [722.0]},
  {'dimensions': [{'name': '2024-01-09'}], 'metrics': [714.0]},
  {'dimensions': [{'name': '2024-01-04'}], 'metrics': [713.0]},
  {'dimensions': [{'name': '2024-01-03'}], 'metrics': [640.0]},
  {'dimensions': [{'name': '2024-01-05'}], 'metrics': [632.0]},
  {'dimensions': [{'name': '2024-01-07'}], 'metrics': [620.0]},
  {'dimensions': [{'name': '2024-01-06'}], 'metrics': [614.0]}],
 'total_rows': 7,
 'total_rows_rounded': False,
 'sampled': False,
 'contains_sensitive_data': False,
 'sample_share': 1.0,
 'sample_size': 4650,
 'sample_sp

In [14]:
json_data['data']

[{'dimensions': [{'name': '2024-01-08'}], 'metrics': [722.0]},
 {'dimensions': [{'name': '2024-01-09'}], 'metrics': [714.0]},
 {'dimensions': [{'name': '2024-01-04'}], 'metrics': [713.0]},
 {'dimensions': [{'name': '2024-01-03'}], 'metrics': [640.0]},
 {'dimensions': [{'name': '2024-01-05'}], 'metrics': [632.0]},
 {'dimensions': [{'name': '2024-01-07'}], 'metrics': [620.0]},
 {'dimensions': [{'name': '2024-01-06'}], 'metrics': [614.0]}]

In [15]:
[(i['dimensions'][0]['name'], 
  i['metrics'][0]) for i in json_data['data']]

[('2024-01-08', 722.0),
 ('2024-01-09', 714.0),
 ('2024-01-04', 713.0),
 ('2024-01-03', 640.0),
 ('2024-01-05', 632.0),
 ('2024-01-07', 620.0),
 ('2024-01-06', 614.0)]

In [17]:
#   загружаем яндекс.метрику в датафрейм

y_df = pd.DataFrame([(i['dimensions'][0]['name'], 
  i['metrics'][0]) for i in json_data['data']], columns=['date', 
                                                        'visits'])
y_df

,date,visits
0,2024-01-08,722.0
1,2024-01-09,714.0
2,2024-01-04,713.0
3,2024-01-03,640.0
4,2024-01-05,632.0
5,2024-01-07,620.0
6,2024-01-06,614.0


In [18]:
#    загружаем датафрейм в Google Docs

spreadsheet_name = 'A new spreadsheet'
sheet = 'Yandex_visits'
d2g.upload(y_df, spreadsheet_name, sheet, credentials=credentials, row_names=True)

<Worksheet 'Yandex_visits' id:1450629507>

In [ ]:
#   